# 命名实体识别

### 导入必要的包

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import os
import ast


### 设置使用的cuda设备

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "8"
CUDA_VISIBLE_DEVICES="8"


### 命名实体识别函数

In [5]:
def predict(messages, model, tokenizer, device):
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512)
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
    unique_data = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return unique_data

### 结果去重函数

In [6]:
def remove_duplicates(data):
    seen = set()
    unique_data = []
    for item in data:
        item_tuple = (item['label'], item['text'])
        if item_tuple not in seen:
            seen.add(item_tuple)
            unique_data.append(item)
    return unique_data

### 加载模型权重

In [7]:
tokenizer = AutoTokenizer.from_pretrained("/home/yuwenhan/law-LLM/buaa&zgzf/finetune/ZhipuAI/glm-4-9b-chat/", use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("/home/yuwenhan/law-LLM/buaa&zgzf/finetune/ZhipuAI/glm-4-9b-chat/", device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 10/10 [00:17<00:00,  1.79s/it]


### 加载微调权重

In [9]:
model = PeftModel.from_pretrained(model, model_id="/home/yuwenhan/law-LLM/buaa&zgzf/finetune/output/GLM4-NER-2/checkpoint-200")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

### 设置微调指令

In [10]:
instruction = ("你是一个法律命名实体识别的专家。请根据给定文本，从以下十个方面（犯罪嫌疑人、受害人、被盗货币、物品价值、"
               "盗窃获利、被盗物品、作案工具、时间、地点、组织机构）提取文中的实体，没有用None表示，并按照以下格式返回结果："
               "[犯罪嫌疑人: xxx; 受害人： xxx; 被盗货币： None; ……]")


### 使用用户输入

In [11]:
input_value = input("请输入文本内容: ")

### 和并最终输入

In [12]:
messages = [
    {"role": "system", "content": instruction},
    {"role": "user", "content": f"{input_value}"}
]

### 生成结果并去重

In [13]:
response = predict(messages, model, tokenizer, device)
data_list = ast.literal_eval(response)
unique_data = remove_duplicates(data_list)


### 输出

In [14]:
print(unique_data)

[{'label': '犯罪嫌疑人', 'text': '李小明'}, {'label': '受害人', 'text': '张强'}, {'label': '时间', 'text': '2024年6月15日'}, {'label': '地点', 'text': '北京市朝阳区三里屯'}, {'label': '组织机构', 'text': '北京市人民检察院朝阳区分院'}, {'label': '组织机构', 'text': '北京市朝阳区人民法院'}, {'label': '组织机构', 'text': '北京市华龙律师事务所'}]


In [26]:
import pandas as pd
# Convert the output to a table
def convert_to_table(response):
    df = pd.DataFrame([response])
    return df

# Use the function to convert response to table
df = convert_to_table(unique_data)
if df is not None:
    print(df)

                                   0                               1  \
0  {'label': '犯罪嫌疑人', 'text': '李小明'}  {'label': '受害人', 'text': '张强'}   

                                       2  \
0  {'label': '时间', 'text': '2024年6月15日'}   

                                      3  \
0  {'label': '地点', 'text': '北京市朝阳区三里屯'}   

                                            4  \
0  {'label': '组织机构', 'text': '北京市人民检察院朝阳区分院'}   

                                         5  \
0  {'label': '组织机构', 'text': '北京市朝阳区人民法院'}   

                                         6  
0  {'label': '组织机构', 'text': '北京市华龙律师事务所'}  
